# GAN

# density estimation
    조사한 데이터(연속형 변수)가 x1,x2,x3라면 x1,x2,x3을 각각 평균으로하는 가우시안 분포를 세개 그림(데이터가 세개가 아니면 가우시안 분포가 그 개수만큼 나오고 그것들을 중첩한 새로운 분포를 형성) 
	-> 그러면 여러개의 국소적 최빈값을 가지는 분포가 형성됨.
	unseen data에 대해서는 만들어놓은 분포에서 y값을 가져오면 됨(무한한 개수의 샘플을 얻어올 수 있음).
	데이터가 벡터이면 x축은 벡터의 차원만큼 생기게 된다.
	예를들어 64*64*3의 이미지에 대해 density estimation을 하여 확률분포를 구하고 그 분포에서 새로운 데이터를 뽑아낸다면 그 데이터 또한 64*64*3의 이미지일 것.
	이것이 생성모델의 output이 됨.
	GAN을 학습하면 내부적으로 density estimation을 진행하고 랜덤으로 이미지를 생성하면 분포에서 최빈값들 중 하나로 될 가능성이 높을 것임.
		-> 꽤 그럴싸한 사람얼굴 이미지가 생성됨 (사람 얼굴 이미지 데이터라면)

# GAN 동작과정
    
	Discriminant 는 입력이미지를 시그모이드를 통해 0~1로 분류하고 1에 가까울수록 real image, 0에 가까울수록 fake image라고 classification한다.
	real image에 대해서는 discriminant가 label을 1로, G가 만든 image에 대해서는 0으로 학습함.
	G는 random vector인 latent code Z가 촉발하여 image를 생성하는데 이것을 discriminant가 1로 학습하게끔 계속해서 학습한다.
	discriminant의 입장에서는 maximize log(D(x))+log(1-D(G(z)))이고(최대는 0) G입장에서는 minimize log(1-D(G(z)))임(최소는 -무한대) (cost function은 binary cross entropy loss(real or fake)을 쓴다)
	G와 D의 파라미터 각각을 gradient descent함
	코드로 보면 G에 대한 gradient를 갱신할 때 minimize log(1-D(G(z)))를 안하고 maximize log(D(G(z)))를 하는 이유는 (실제로는 크로스 엔트로피 사용)
	D에 의해서 g의 d(G(z)) 값이 상당히 0에 가깝게 생성될 것임. g입장에서는 이것을 1로 보내야하는데 그러기 위해서는 기울기가 상당히 가팔라야 빨리 이동이 가능한데 log(1-x) 그래프는 x=0에서기울기가 완만해서 갈길이 너무 멂.
	따라서 gradient descent을 다른 criteria를 사용해서 따로 진행하는 것이 효율적 -> -log(x)그래프 사용 -> Generator의 loss = -1/m 시그마 log(D(G(z)))

# DCGAN
    GAN에 convolution 적용한 모델	
	deconvolution: conv를 역과정으로 한 것으로 input 값을 필터의 픽셀 하나하나에 곱해서 output으로 내보내는 것.
	checkerboard artifact: stride가 필터의 가로세로보다 작으면 특정 위치는 곱셈이 여러번 진행되므로 이것이 checkerboard의 구조를 띤다고해서 붙여진 이름. 학습 초에 나타나는 경향
	최근에는 deconvolution을 안하고 conv를 하되, 가로세로 사이즈를 그대로 유지하고(padding으로) 나중에 사이즈를 키우는 과정을 별도로 진행하는 식으로 decoding하는 추세임
	leaky ReLU: 0보다 작은 구간에서는 0이 되어 back prop시 사라지는데 그것을 방지하고자 입력이 0보다 작을 때 기울기를 어느정도 살려줌 

# conditional GAN
    latent code와 더불어 메타 데이터인 C를 input 으로 받아서 G가 이미지를 생성한다. C는 이미지에 대한 추가적인 정보로 인위적으로 이미지를 다룰 기회를 준다. 예를 들어 이미지는 남자다 와 같은 정보.
	C는 discriminant에도 전달돼서 G가 만든 이미지가 real/fake 구분뿐만 아니라 이것이 남자인지 아닌지도 학습. C에 입력된 값은 남자인데 real image의 label을 여자라고 하면 D는 이것을 fake로 판단하며 학습이 진행된다.


# ACGAN
    ACGAN은 D가 C를 맞히게끔 학습하는 것. 위의 cGAN처럼 real image에 따로 label은 설정하지 않고 이미지가 지닌 C의 정보를 classification하도록 학습. softmax를 통해 10 20 30대의 연령대를 맞히게 하는 등.
	G가 생성한 fake image가 남자라는 C를 지녔고 real image에서 D가 판단한 C가 여자가 80% 남자가 20%라면 back prop에서 G에 대한 파라미터를 갱신하여 더 좋은 이미지를 생성하는 것은 맞지만, 
	D에서 G를 속이기 위해 일부러 여자를 더 높게 가는 등의 back prop은 진행하지 않는다. D는 단지 G의 update를 위한 back prop의 경로이고 update는 하지 않음.
	D에서는 단지 real image만 보고 C를 classify하면 된다.